# Grouping Data & Aggregating Variables

## 4.8 Task

In [1]:
#import libraries
import pandas as pd
import numpy as np
import os

#### Step 1.

In [2]:
#import dataset
path = r'C:\Users\Kid\01.2024 Instacart Basket Analysis Project'
df_ords_prods = pd.read_pickle(os.path.join(path, '02. Data', 'Prepared Data', 'orders_products_derived.pkl'))

In [3]:
df_ords_prods.head()

order_id  user_id  order_number  orders_day_of_week  order_hour_of_day  \
0   2539329        1             1                   2                  8   
1   2398795        1             2                   3                  7   
2    473747        1             3                   3                 12   
3   2254736        1             4                   4                  7   
4    431534        1             5                   4                 15   

   days_since_prior_order  first_order  product_id  add_to_cart_order  \
0                     NaN         True         196                  1   
1                    15.0        False         196                  1   
2                    21.0        False         196                  1   
3                    29.0        False         196                  1   
4                    28.0        False         196                  1   

   reordered product_name  aisle_id  department_id  prices        price_range  \
0          0         Soda        77              7     9.0  Mid-range product   
1          1         Soda        77              7     9.0  Mid-range product   
2          1         Soda        77              7     9.0  Mid-range product   
3          1         Soda        77              7     9.0  Mid-range product   
4          1         Soda        77              7     9.0  Mid-range product   

      busiest_day busiest_days busiest_period_of_day  
0  Regularly busy  Regular day        Average orders  
1  Regularly busy  Slowest day        Average orders  
2  Regularly busy  Slowest day           Most orders  
3      Least busy  Slowest day        Average orders  
4      Least busy  Slowest day           Most orders

#### Step 2. & 3

In [4]:
#finding aggregated mean of 'order_number' grouped by 'department_id'
df_ords_prods.groupby('department_id')['order_number'].mean().sort_values(ascending = False)

department_id
21    22.902379
10    20.197148
18    19.310397
4     17.811403
16    17.665606
2     17.277920
7     17.225802
19    17.177343
3     17.170395
14    16.773669
13    16.583536
20    16.473447
6     16.439806
11    16.170638
15    16.165037
9     15.895474
12    15.887671
17    15.694469
1     15.457838
8     15.340650
5     15.215751
Name: order_number, dtype: float64

##### The aggregated result of the subset produced the average orders of 8 departments only instead of all departments. Additonally, the average 'order_number' in the subset shows that 'department_id 4' has an average of around 19 orders, while 'department_id 17' has an average of 11 orders -- the least of all other departments. 

##### Aggregating the entire dataframe produced obviously a different, but rather more accurate results. Here, the data shows the average order number of all 21 departments instead of only 8. Subsequently, 'department_id 5' has now the least (approx. 15) average order number, while 'department_id 21' averages the most (approx. 22) number of orders of all other departments.

#### Step 4.

In [5]:
#deriving loyalty flag column
df_ords_prods['max_order'] = df_ords_prods.groupby(['user_id'])['order_number'].transform(np.max)

C:\Users\Kid\AppData\Local\Temp\ipykernel_19784\1787993493.py:2: FutureWarning: The provided callable <function amax at 0x000001F04D0042C0> is currently using SeriesGroupBy.max. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "max" instead.
  df_ords_prods['max_order'] = df_ords_prods.groupby(['user_id'])['order_number'].transform(np.max)


In [6]:
df_ords_prods.loc[df_ords_prods['max_order'] > 40, 'loyalty_flag'] = 'Loyal customer'

In [7]:
df_ords_prods.loc[(df_ords_prods['max_order'] <= 40) & (df_ords_prods['max_order'] > 10), 'loyalty_flag'] = 'Regular customer'

In [8]:
df_ords_prods.loc[df_ords_prods['max_order'] <= 10, 'loyalty_flag'] = 'New customer'

In [9]:
#checking output
df_ords_prods[['user_id', 'max_order', 'loyalty_flag']].head(20)

user_id  max_order      loyalty_flag
0         1         10      New customer
1         1         10      New customer
2         1         10      New customer
3         1         10      New customer
4         1         10      New customer
5         1         10      New customer
6         1         10      New customer
7         1         10      New customer
8         1         10      New customer
9         1         10      New customer
10       15         22  Regular customer
11       15         22  Regular customer
12       15         22  Regular customer
13       15         22  Regular customer
14       15         22  Regular customer
15       19          9      New customer
16       19          9      New customer
17       19          9      New customer
18       21         33  Regular customer
19       31         20  Regular customer

In [10]:
#checking values frequency
df_ords_prods['loyalty_flag'].value_counts(dropna = False)

loyalty_flag
Regular customer    15876776
Loyal customer      10284093
New customer         6243990
Name: count, dtype: int64

#### Step 5.

In [11]:
#checking product 'prices' statistics grouped by 'loyalty_flag'
df_ords_prods.groupby('loyalty_flag').agg({'prices' : ['min', 'max', 'mean']})

prices                    
                    min      max       mean
loyalty_flag                               
Loyal customer      1.0  99999.0  10.386336
New customer        1.0  99999.0  13.294670
Regular customer    1.0  99999.0  12.495717

##### The data shows that the average product prices for loyal customers differs slightly from new and regular customers. Loyal customers has the least average product prices (approx. 10.40) while new customers has the highest average product prices (approx. 13.30). The average product prices (approx.12.50) for regular customers sits in between that of new and loyal customers.


#### Step 6.

In [12]:
#deriving spending flag column
df_ords_prods['average_price_spent'] = df_ords_prods.groupby(['user_id'])['prices'].transform(np.mean)

C:\Users\Kid\AppData\Local\Temp\ipykernel_19784\3174461219.py:2: FutureWarning: The provided callable <function mean at 0x000001F04D004FE0> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df_ords_prods['average_price_spent'] = df_ords_prods.groupby(['user_id'])['prices'].transform(np.mean)


In [13]:
df_ords_prods.loc[df_ords_prods['average_price_spent'] >= 10, 'spending_flag'] = 'High spender'

In [14]:
df_ords_prods.loc[df_ords_prods['average_price_spent'] < 10, 'spending_flag'] = 'Low spender'

In [15]:
#checking output
df_ords_prods[['user_id', 'average_price_spent', 'spending_flag']].head(20)

user_id  average_price_spent spending_flag
0         1             6.367797   Low spender
1         1             6.367797   Low spender
2         1             6.367797   Low spender
3         1             6.367797   Low spender
4         1             6.367797   Low spender
5         1             6.367797   Low spender
6         1             6.367797   Low spender
7         1             6.367797   Low spender
8         1             6.367797   Low spender
9         1             6.367797   Low spender
10       15             3.980556   Low spender
11       15             3.980556   Low spender
12       15             3.980556   Low spender
13       15             3.980556   Low spender
14       15             3.980556   Low spender
15       19             8.372059   Low spender
16       19             8.372059   Low spender
17       19             8.372059   Low spender
18       21             7.012683   Low spender
19       31             8.137793   Low spender

In [16]:
#checking 'average_price_spent' values frequency
df_ords_prods['average_price_spent'].value_counts(dropna = False)

average_price_spent
8.000000     10337
7.500000      9116
8.100000      8254
7.750000      7048
7.400000      7044
             ...  
13.800000        3
10.266667        3
2.700000         3
5.600000         3
12.200000        3
Name: count, Length: 128637, dtype: int64

In [17]:
#checking 'spending_flag' values frequency
df_ords_prods['spending_flag'].value_counts(dropna = False)

spending_flag
Low spender     31770614
High spender      634245
Name: count, dtype: int64

#### Step 7.

In [18]:
#deriving regularity flag column
df_ords_prods['orders_frequency'] = df_ords_prods.groupby(['user_id'])['days_since_prior_order'].transform(np.median)

C:\Users\Kid\AppData\Local\Temp\ipykernel_19784\1557903422.py:2: FutureWarning: The provided callable <function median at 0x000001F04D14DEE0> is currently using SeriesGroupBy.median. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "median" instead.
  df_ords_prods['orders_frequency'] = df_ords_prods.groupby(['user_id'])['days_since_prior_order'].transform(np.median)


In [19]:
df_ords_prods.loc[df_ords_prods['orders_frequency'] > 20, 'regularity_flag'] = 'Non-frequent customer'

In [20]:
df_ords_prods.loc[(df_ords_prods['orders_frequency'] > 10) & (df_ords_prods['orders_frequency'] <= 20), 'regularity_flag'] = 'Regular customer'

In [21]:
df_ords_prods.loc[df_ords_prods['orders_frequency'] <= 10, 'regularity_flag'] = 'Frequent customer'

In [22]:
df_ords_prods['regularity_flag'].value_counts(dropna=False)

regularity_flag
Frequent customer        21559853
Regular customer          7208564
Non-frequent customer     3636437
nan                             5
Name: count, dtype: int64

In [23]:
#checking output
df_ords_prods[['user_id', 'orders_frequency', 'regularity_flag']].head(20)

user_id  orders_frequency        regularity_flag
0         1              20.5  Non-frequent customer
1         1              20.5  Non-frequent customer
2         1              20.5  Non-frequent customer
3         1              20.5  Non-frequent customer
4         1              20.5  Non-frequent customer
5         1              20.5  Non-frequent customer
6         1              20.5  Non-frequent customer
7         1              20.5  Non-frequent customer
8         1              20.5  Non-frequent customer
9         1              20.5  Non-frequent customer
10       15              10.0      Frequent customer
11       15              10.0      Frequent customer
12       15              10.0      Frequent customer
13       15              10.0      Frequent customer
14       15              10.0      Frequent customer
15       19               6.0      Frequent customer
16       19               6.0      Frequent customer
17       19               6.0      Frequent customer
18       21               7.5      Frequent customer
19       31               6.0      Frequent customer

In [24]:
#checking 'orders_frequency' value frequency
df_ords_prods['orders_frequency'].value_counts(dropna = False)

orders_frequency
7.0     5682399
6.0     2983604
8.0     2713964
5.0     2486061
4.0     2314289
         ...   
26.5       5514
29.5       5095
1.5        1684
0.5         235
NaN           5
Name: count, Length: 62, dtype: int64

In [30]:
df_ords_prods[df_ords_prods['orders_frequency'].isnull()]

order_id  user_id  order_number  orders_day_of_week  \
13645692    895835   159838             1                   0   
17251990    895835   159838             1                   0   
17622767    895835   159838             1                   0   
24138593    895835   159838             1                   0   
25880002    895835   159838             1                   0   

          order_hour_of_day  days_since_prior_order  first_order  product_id  \
13645692                 17                     NaN         True       10749   
17251990                 17                     NaN         True       33401   
17622767                 17                     NaN         True       23695   
24138593                 17                     NaN         True       21334   
25880002                 17                     NaN         True       22198   

          add_to_cart_order  reordered  ...        price_range  busiest_day  \
13645692                  3          0  ...  Mid-range product  Busiest day   
17251990                  6          0  ...  Mid-range product  Busiest day   
17622767                  2          0  ...  Low-range product  Busiest day   
24138593                  5          0  ...  Mid-range product  Busiest day   
25880002                  1          0  ...  Low-range product  Busiest day   

          busiest_days  busiest_period_of_day max_order  loyalty_flag  \
13645692   Busiest day            Most orders         1  New customer   
17251990   Busiest day            Most orders         1  New customer   
17622767   Busiest day            Most orders         1  New customer   
24138593   Busiest day            Most orders         1  New customer   
25880002   Busiest day            Most orders         1  New customer   

         average_price_spent spending_flag  orders_frequency  \
13645692                7.42   Low spender               NaN   
17251990                7.42   Low spender               NaN   
17622767                7.42   Low spender               NaN   
24138593                7.42   Low spender               NaN   
25880002                7.42   Low spender               NaN   

                regularity_flag  
13645692  Non-frequent customer  
17251990  Non-frequent customer  
17622767  Non-frequent customer  
24138593  Non-frequent customer  
25880002  Non-frequent customer  

[5 rows x 24 columns]

#### Null values in the 'orders_frequency' flag column correspond to 1 user with only 1 order.

In [28]:
#handling nulls
df_ords_prods.loc[df_ords_prods['orders_frequency'].isna(), 'regularity_flag'] = 'Non-frequent customer'

#### Step 8.

In [ ]:
df_ords_prods.to_pickle(os.path.join(path, '02. Data', 'Prepared Data', 'orders_products_aggregated.pkl'))